In [2]:
#importing required libraries
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [4]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True, #importing dataset, no augmentation used for this model
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),   # priorly calculated mean and standard deviations.
                             (0.2470, 0.2435, 0.2616))
    ]))


Extracting ../data-unversioned/p1ch6/cifar-10-python.tar.gz to ../data-unversioned/p1ch6/


In [5]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([               #loading the val set
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [6]:
class resblock(nn.Module):          #creating a resblock which would be used iterativealy.
    def __init__(self, n):
        super(resblock, self).__init__()
        self.conv = nn.Conv2d(n, n, kernel_size=3,         #defining conv and batch norm, so they may be used again in forward simply if required.
                              padding=1, bias=False)  
        self.batch_norm = nn.BatchNorm2d(num_features=n)

    def forward(self, x):                        #Actual resblock with the skip connection over 1 layer
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [7]:
class deep(nn.Module):                   # the model which uses resblock iteratively and some other layers
    def __init__(self, n1=32, n_blocks=10):
        super().__init__()
        self.n1 = n1
        self.conv1 = nn.Conv2d(3, n1, kernel_size=3, padding=1)        #defining all the layers which may be required
        self.resblocks = nn.Sequential(
            *(n_blocks * [resblock(n=n1)]))
        self.fc1 = nn.Linear(8 * 8 * n1, 32)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)            # The architecture which is used
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [8]:

device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))                                  # shifting to gpu if available
print(f"Training on device {device}.")

Training on device cuda.


In [9]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):     # defining the training loop
    for epoch in range(1, n_epochs + 1):                                   # the inputs for the function will be defined in the next cell
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # shifting stuff to gpu
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()           # making grads zero so they dont accumalate
            loss.backward()                 # backprop
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:                  # printing loss at intervals
            print('{} Epoch {}, Training loss {}'.format(           
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [10]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,      
                                           shuffle=True)              # defining mini batch size and shuffling

model =  deep().to(device=device)  #shifting model to gpu
optimizer = optim.SGD(model.parameters(), lr=1e-2)       # defining optimizer and lossfn
loss_fn = nn.CrossEntropyLoss()

In [ ]:
training_loop(  
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)            #calling the training loop finally

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


2021-07-28 12:46:48.611952 Epoch 1, Training loss 2.3159351675101862
2021-07-28 12:48:51.426712 Epoch 10, Training loss 1.2141340057868177
2021-07-28 12:51:07.649465 Epoch 20, Training loss 0.8525501969449051
2021-07-28 12:53:23.922241 Epoch 30, Training loss 0.7394904084598927
2021-07-28 12:55:43.709836 Epoch 40, Training loss 0.6055725093578439
2021-07-28 12:57:58.111331 Epoch 50, Training loss 0.5347025550882835
2021-07-28 13:00:12.421280 Epoch 60, Training loss 0.49188517281771316
2021-07-28 13:02:29.205341 Epoch 70, Training loss 0.4443377943333153
2021-07-28 13:04:44.428484 Epoch 80, Training loss 0.4130946373009621
2021-07-28 13:07:01.022181 Epoch 90, Training loss 0.3808805112872282
2021-07-28 13:09:15.847943 Epoch 100, Training loss 0.3639952096888019


In [ ]:

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,                 # our model is trained, no shuffling required now
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,               
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:               
        correct = 0
        total = 0

        with torch.no_grad():  # we do no need backpropagation for this hence the nograd
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # taking the class at maximum probability as prediction
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # calculating the no. of correct predictions.

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.87
Accuracy val: 0.69


In [ ]:
numel_list = [p.numel() for p in model.parameters()]      #checking no. of parameters
sum(numel_list), numel_list

(76074, [864, 32, 9216, 32, 32, 65536, 32, 320, 10])